Программа предназначена для вычисления интегралов с помощью квадратурных формул Гаусса (КФ Гаусса). Основные этапы включают определение узлов и коэффициентов КФ Гаусса для различных значений N, проверку точности на многочленах высоких степеней и вычисление интегралов с заданными параметрами.

Импорт библиотек:
Импортируются библиотеки math и scipy.integrate для математических вычислений и интегрирования.

In [1]:
import math
import scipy.integrate as spi

Определение константы точности:
EPS задаёт точность для метода секущих, используемого при поиске корней многочлена Лежандра.

In [2]:
EPS = 10 ** - 12

Функция localize_roots в коде представляет собой ключевой элемент для определения интервалов, на которых могут находиться корни многочлена Лежандра. Это необходимо для дальнейшего их точного нахождения с помощью метода бисекции или метода секущих. Вот более детальное описание этой функции:

In [3]:
def localize_roots(a, b, f, n):
    roots_sections = []
    h = (b - a) / n

    beg, end = a, a + h
    while end <= b:
        y1, y2 = f(beg), f(end)
        if y1 * y2 <= 0:
            roots_sections.append((beg, end))
        beg, end = end, end + h
    return roots_sections

Функции для многочленов:
Определены функции f_5, f_7, f_9, представляющие многочлены различных степеней, используемые для проверки точности квадратурных формул Гаусса.

In [4]:
def f_5(x):
    return x ** 5 + x ** 3 + 1


def f_7(x):
    return x ** 7 + x ** 5 + 1


def f_9(x):
    return x ** 9 + x ** 7 + 1

Функция f:
Функция f определяет подынтегральное выражение

In [5]:
def f(x):
    return math.cos(x ** 2)

Функция p_n:
p_n представляет собой функцию для вычисления многочленов Лежандра.

In [6]:
def p_n(x, n):
    match n:
        case 0:
            return 1
        case 1:
            return x
        case _:
            return (2 * n - 1) / n * p_n(x, n - 1) * x - (n - 1) / n * p_n(x, n - 2)

Метод бисекции:
bisection_method реализует метод бисекции для нахождения корней многочлена Лежандра.

In [7]:


def bisection_method(f, a, b):
    roots = []
    intervals = []
    h = (b - a) / 100
    x1 = a
    x2 = x1 + h
    y1 = f(x1)
    while x2 <= b:
        y2 = f(x2)
        if y1 * y2 <= 0:
            intervals.append((x1, x2))
        x1 = x2
        x2 += h
        y1 = y2

    for a, b in intervals:
        while b - a > 2 * 10 ** (-13):
            c = (a + b) / 2
            if f(a) * f(c) <= 0:
                b = c
            else:
                a = c
        roots.append((a + b) / 2)
    l = len(roots)
    if l % 2 != 0:
        roots[int(l / 2)] = 0
    return roots

Вычисление узлов и коэффициентов КФ Гаусса:
Основной цикл программы вычисляет узлы и коэффициенты КФ Гаусса для значений N от 1 до 8. Используется метод секущих для нахождения корней многочлена Лежандра и последующего вычисления коэффициентов.

Проверка точности КФ Гаусса:
Для N от 3 до 5 производится проверка точности КФ Гаусса на соответствующих многочленах высокой степени.

In [8]:
print("Вычисление интегралов при помощи КФ Гаусса. Вариант 8. [𝑎, 𝑏] = [0, pi/4], 𝑓(𝑥) = cos(𝑥^2), 𝑁 = 6,7,8")
gauss_n = {}
check_polys = {3: f_5, 4: f_7, 5: f_9}
for n in range(1, 9):
    print("\nN =", n)
    sections = localize_roots(-1, 1, lambda x: p_n(x, n), n)
    print("{:<20} {:<20}".format("Узел", "Коэффициент"))
    nodes_t = bisection_method(lambda x: p_n(x, n), -1, 1)
    c_k_list = []
    for t in nodes_t:
        c_k = 2 * (1 - t ** 2) / (n ** 2 * p_n(t, n - 1) ** 2)
        c_k_list.append(c_k)
        print("{:<20} {:<20}".format(t, c_k))
    kf_gauss_init = lambda func: sum(c_k_list[i - 1] * func(nodes_t[i - 1]) for i in range(n))
    if n in range(3, 6):
        print(f"\nПроверка для x ** {2 * n - 1} + x ** {2 * n - 3} + 1")
        real_val, _ = spi.quad(check_polys[n], -1, 1)
        res = kf_gauss_init(check_polys[n])
        print("Значение интеграла по отрезку [-1, 1] в библиотеке scipy:", real_val)
        print(f"Приближенное значение КФ Гаусса ({n} узлов): {res:.15f}")
        print("Абсолютная фактическая погрешность:", abs(res - real_val))
    gauss_n[n] = (nodes_t, c_k_list)

Вычисление интегралов при помощи КФ Гаусса. Вариант 8. [𝑎, 𝑏] = [0, pi/4], 𝑓(𝑥) = cos(𝑥^2), 𝑁 = 6,7,8

N = 1
Узел                 Коэффициент         
0                    2.0                 

N = 2
Узел                 Коэффициент         
-0.5773502691895551  1.000000000000367   
0.5773502691895562   1.0000000000003613  

N = 3
Узел                 Коэффициент         
-0.7745966692415093  0.5555555555553325  
0                    0.8888888888888888  
0.7745966692415104   0.5555555555553231  

Проверка для x ** 5 + x ** 3 + 1
Значение интеграла по отрезку [-1, 1] в библиотеке scipy: 2.0
Приближенное значение КФ Гаусса (3 узлов): 1.999999999999540
Абсолютная фактическая погрешность: 4.602984660095899e-13

N = 4
Узел                 Коэффициент         
-0.8611363115940913  0.34785484513700454 
-0.3399810435849207  0.6521451548623846  
0.3399810435849219   0.6521451548623816  
0.8611363115940924   0.34785484513699166 

Проверка для x ** 7 + x ** 5 + 1
Значение интеграла по отрезку [-1

Вычисление интегралов:
В завершающей части кода пользователь вводит значения N1 N2 N3

и границы интегрирования [a b]

Для этих параметров производится вычисление интегралов с использованием КФ Гаусса.

In [9]:
print("\n")
print("Вычисление интегралов при помощи КФ Гаусса. [𝑎, 𝑏] = [0, pi/4], 𝑓(𝑥) = cos(𝑥^2), 𝑁 = 6,7,8")
a, b = 0, math.pi / 4
while True:
    n_s = map(int, input("Введите N1, N2, N3 через пробел: ").split())
    for n in n_s:
        if n not in gauss_n.keys():
            sections = lab1.localize_roots(-1, 1, lambda x: p_n(x, n), n)
            nodes_t = bisection_method(lambda x: p_n(x, n), -1, 1)
            c_k_list = [2 * (1 - t ** 2) / (n ** 2 * p_n(t, n - 1) ** 2) for t in nodes_t]
            gauss_n[n] = (nodes_t, c_k_list)
        t_list, c_k_list = gauss_n[n]
        a_k_list, x_list = [], []
        print("{:<20} {:<20}".format("Узел", "Коэффициент для [a, b]"))
        for i in range(n):
            a_k = (b - a) / 2 * c_k_list[i]
            x_k = (b - a) / 2 * t_list[i] + (b + a) / 2
            a_k_list.append(a_k)
            x_list.append(x_k)
            print("{:<20} {:<20}".format(x_k, a_k))
        kf_gauss = lambda f: sum(a_k_list[j] * f(x_list[j]) for j in range(n))

        real_val, _ = spi.quad(f, a, b)
        res = kf_gauss(f)
        print("Значение интеграла по отрезку [a, b] в библиотеке scipy:", real_val)
        print(f"Приближенное значение КФ Гаусса ({n} узлов): {res:.15f}")
        print("Абсолютная фактическая погрешность:", abs(res - real_val))
        print("\n")

    a, b = float(input("Введите границу интегрирования a: ")), float(input("Введите границу интегрирования b: "))




Вычисление интегралов при помощи КФ Гаусса. Вариант 8. [𝑎, 𝑏] = [0, pi/4], 𝑓(𝑥) = cos(𝑥^2), 𝑁 = 6,7,8
Введите N1, N2, N3 через пробел: 6 7 8
Узел                 Коэффициент для [a, b]
0.026519159759078037 0.06727897082957879 
0.13304276282283567  0.14167073844813452 
0.2989935464481703   0.18374937242075673 
0.4864046169492785   0.18374937242075595 
0.652355400574613    0.14167073844813133 
0.7588790036383708   0.06727897082956964 
Значение интеграла по отрезку [a, b] в библиотеке scipy: 0.7560352774629175
Приближенное значение КФ Гаусса (6 узлов): 0.756035277462932
Абсолютная фактическая погрешность: 1.3988810110276972e-14


Узел                 Коэффициент для [a, b]
0.019985276088711035 0.05084862730793517 
0.10150046606288216  0.10984005038406841 
0.23332406344095474  0.1499443101983797  
0.39269908169872414  0.1641321876161198  
0.552074099956494    0.14994431019837823 
0.6838976973345666   0.10984005038406439 
0.7654128873087378   0.050848627307924846
Значение интеграла по отр

ValueError: could not convert string to float: 'o'